## 2.0 Test the machine learning model

Here we collect some images that were never used for training and/or validation of the machine learning models. We process those images using
1. Noise reduction model: To reduce overall noise in the image
2. Super-resolution model: To increase the pixel count per unit area by 4X
The result of the noise reduction model is fed as an input to the super-resolution model
Finally, we reduce the super resolved image to its original size using cubic-spline interpolation. This down-sampling further increases the signal to noise ratio (40% for every 2X down-sampling)

In [ ]:
import os
import glob
import cv2
import skimage.io
import numpy as np

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
model_NR = load_model("D:\\myData\\ML_Quality\\autoencoder_NR.h5")
model_SR = load_model("D:\\myData\\ML_Quality\\super_resolution_SR.h5")

In [ ]:
test_dataPath = "D:\\myData\\ML_Quality\\x_test\\*.tif"
result_path = "D:\\myData\\ML_Quality\\x_test\\ML_NRSR"

The code below is writte inside a for loop so that multiple images are read and tested

In [ ]:
files = glob.glob(test_dataPath)

#### 2.1 Process the images

Here we implemented three similar functions. 
First one, uses only noise reduction model and demonstrates the effect of this model on a given image.
Second one, uses the super resolution model and scales the image 4X without blurring the image and with additional enhancements to contrast, etc. as learned by training process
In the third implementation we concatenated noise reduction and super resolution processes to take advantage of multiple models to improve the image quality.
Finally we bring the super resolved image back to original image size using cubic spline down-sampling there by further increasing the signal to noise ratio.

#### Only noise reduction

In [ ]:
for file in files:   
    x_NR_test=[] # list to hold input image
    #read the image file
    img = skimage.io.imread(file)      
    img = np.array(img)
    
    dim=img.shape # used to down size the image back to original size after super-resolution
    
    #pad the image to reduce boundary issues
    pad = 32
    img = np.pad(img, pad, mode="reflect")
    x_NR_test.append(img)
    
    ## do the necessary input data shape adjustments 9as required by tensorflow)
    x_NR_test = np.dstack(x_NR_test)
    x_NR_test = np.rollaxis(x_NR_test,-1)
    x_NR_test = x_NR_test[:,:,:, tf.newaxis]
    
    # do the noise reduction using NR model
    NR_predict = model_NR(x_NR_test)
    
    #use the noise reduced image to build super-resolution image
    img = predictions[0]
    img = img[:, :, 0] # back to 2D, 1 channel image
    img = img[pad:-pad, pad:-pad] # unpad the image
    
    # byte scale the image (0-255)
    img = (255*(img-np.min(img))/(np.max(img) - np.min(img)))
    
    #write the output
    skimage.io.imsave(result_path+"\\NR_"+os.path.basename(file), img)

#### Only super-resolution

In [ ]:
for file in files:   
    x_NR_test=[] # list to hold input image
    #read the image file
    img = skimage.io.imread(file)      
    img = np.array(img)
    
    dim=img.shape # used to down size the image back to original size after super-resolution
    
    #pad the image to reduce boundary issues
    pad = 32
    img = np.pad(img, pad, mode="reflect")
    x_SR_test.append(img)
    
    ## do the necessary input data shape adjustments 9as required by tensorflow)
    x_SR_test = np.dstack(x_SR_test)
    x_SR_test = np.rollaxis(x_SR_test,-1)
    x_SR_test = x_SR_test[:,:,:, tf.newaxis]
    
    #do the super-resolution using SR model
    SR_predict = model_SR(x_SR_test)
    
    # write the image output
    img = predictions[0]
    img = img[:, :, 0]
    
    # resize the image to original image dimensions
    img = cv2.resize(img, dim, interpolation=cv2.INTER_CUBIC) 
    
    #byte scale the image (0-255)
    img = (255*(img-np.min(img))/(np.max(img) - np.min(img)))
    
    #write the output
    skimage.io.imsave(result_path+"\\SR_"+os.path.basename(file), img)

### Concatenating Noise Reduction and Super Resolution

In [ ]:
for file in files:   
    x_NR_test=[] # list to hold input image
    #read the image file
    img = skimage.io.imread(file)      
    img = np.array(img)
    
    dim=img.shape # used to down size the image back to original size after super-resolution
    
    #pad the image to reduce boundary issues
    pad = 32
    img = np.pad(img, pad, mode="reflect")
    x_NR_test.append(img)
    
    ## do the necessary input data shape adjustments 9as required by tensorflow)
    x_NR_test = np.dstack(x_NR_test)
    x_NR_test = np.rollaxis(x_NR_test,-1)
    x_NR_test = x_NR_test[:,:,:, tf.newaxis]
    
    # do the noise reduction using NR model
    NR_predict = model_NR(x_NR_test)
    
    #use the noise reduced image to build super-resolution image
    img = predictions[0]
    img = img[:, :, 0] # back to 2D, 1 channel image
    img = img[pad:-pad, pad:-pad] # unpad the image
    
    # byte scale the image (0-255)
    img = (255*(img-np.min(img))/(np.max(img) - np.min(img)))
    
    # append the image to super resolution input list
    x_SR_test.append(img)
    
    # do the necessary list shape adjustments required for tensorflow
    x_SR_test = np.dstack(x_SR_test)
    x_SR_test = np.rollaxis(x_SR_test,-1)
    x_SR_test = x_SR_test[:,:,:, tf.newaxis]
    
    #do the super-resolution using SR model
    SR_predict = model_SR(x_SR_test)
    
    # write the image output
    img = predictions[0]
    img = img[:, :, 0]
    
    # resize the image to original image dimensions
    img = cv2.resize(img, dim, interpolation=cv2.INTER_CUBIC) 
    
    #byte scale the image (0-255)
    img = (255*(img-np.min(img))/(np.max(img) - np.min(img)))
    
    #write the output
    skimage.io.imsave(result_path+"\\"+os.path.basename(file), img)

### Summary

The above code reads images from a given folder, sets-up image into a appropriate dimension list (as required by tensorflow) and uses
1. Noise Reduction model (model_NR) to reduce the noise in the image using U-net based noise reduction technique
2. Super Resolution model (model_SR) to increase the X and Y dimensions of the image by 4X without blurring the image
In the final processing stage, we down-sample the super-resolved image to its original size (i.e. 4X down-sampling). This improves the signal to noise ratio substantially (40% for every 2X down-sizing) and in the end has not changed image dimensions but significantly improved image quality.

### Results of image enhancement using individual models and the concatenation is shown in the final report. Model description and preliminary conclusions on the results is also reported.